# [OUTDATED][MUST RUN] Iterative Deepening

In [1]:
import time
#from alphabeta_noprint import alphabeta
#from ttalphabeta import ttalphabeta
import numpy as np
#from transpositiontable import TranspositionTable

def iterativedeepening(board, timelimit, ntype, p, a=-np.inf, b=np.inf):
    """
    Calls alpha-beta iteratively, starts at shallow depth and increase depth iteratively
    The function will terminate on following conditions:
    EITHER 1) kernel is interuppted OR 2) timeout OR 3) search depth exceeds board empty positions.
    Parameters: 
        board (HexBoard object):
        timelimit (int): search time limit in seconds. SUGGEST testing with timelimit from 1 first
        ntype (str): carry to alphabeta(), refer to alphabeta() docstring
        p (int): carry to alphabeta(), refer to alphabeta() docstring
        a (float): carry to alphabeta(), refer to alphabeta() docstring
        b (float): carry to alphabeta(), refer to alphabeta() docstring
    Ouput: 
        node (dict): {'state', 'depth', 'children', 'type', 'score', 'move'}
    """
    # Initialize
    timeout = time.time() + timelimit  # Define timeout criteria
    depth = 1  # Start with shallow depth
    tt = TranspositionTable()  # Initialize search with empty
    print('USER NOTE: Interrupt the kernel to terminate search. \n')
    
    try:
        while True:
            print(f'[Iteration status] Start iteration at depth {depth} \n')
            # Option: alpha-beta + TT
            (result_node, tt) = ttalphabeta(board=board, depth=depth, ntype=ntype, p=p, a=a, b=b, tt=tt)  # Use TT from previous search to improve efficiency
            # Alternative option: alpha-beta
            #n = alphabeta(board, depth, ntype, p, a, b)
            print(f'[Iteration status] Finish iteration at depth {depth}:', end=" ")
            print(f'Best move at root node = {result_node["move"]} \n')
            if time.time() > timeout:  # WARNING This method is not perfect and only breaks after search completed, may change to raise + class Exception for instant interrupt
                print('[Iteration status] Termination: TIMEOUT \n')
                print(f'[Iteration report] Return result of completed search at depth {depth} \n')
                break
            if depth == len(board.get_allempty()):
                print('[Iteration status] Termination: EXACT SEARCH')
                print(f'[Iteration report] Return result of completed search at depth {depth} \n')
                break
            depth += 1  # Increase depth after one iteration
        
    except KeyboardInterrupt:  # Interrupt kernel, Ctrl+c in console
        print('[Iteration status] Termination: USER INTERRUPT')
        print(f'[Iteration report] Return result of completed search at depth {depth-1} \n')
        pass
    
    finally:
        return (result_node, tt)  # For test only. Also return latest TT, for debug, potential conflict with RepeatGames.py
        #return result # Normal output

# [MUST RUN][OUTDATED] define class TranspositionTable

In [2]:
class TranspositionTable():
    """Contains a dict that uses board state as keys, and stores info about best moves"""
    
    def __init__(self):
        """Constructor, initiate the transposition table"""
        self.table = {}
    
    def is_empty(self):
        """Check if table is empty"""
        return self.table == {}
    
    def store(self, n):
        """Store result node information to TT"""
        key = n['state'].convert_key()
        if key in self.table.keys():  # Board state already exists in TT
            #print('[TT] Found transpositions')
            # Update TT entry
            if n['depth'] >= self.table[key]['depth']:  # Compare search depth
                self.table[key]['depth'] = n['depth']
                self.table[key]['bestmove'] = n['move']
                self.table[key]['score'] = n['score']
                #print('[TT] Updated depth, best move, score in entry')
        else: # Create new TT entry
            value = {'state': n['state'],
                     'depth': n['depth'],
                     'bestmove': n['move'],
                     'score': n['score']}
            key = n['state'].convert_key()
            self.table.update({key: value})
            #print('[TT] Created new entry')
    
    def lookup(self, n, depth):
        """Return look up result from TT"""
        hit = False
        key = n['state'].convert_key()
        if key in self.table.keys():  # Found tranposition in TT
            # Transposition has larger or equal search depth than current search
            if depth <= self.table[key]['depth']: 
                transposition = self.table[key]
                hit = True # Found transposition with useful depth, can return score
                score = transposition['score']
                bestmove = transposition['bestmove']
                return (hit, score, bestmove)
            # Transposition has smaller search depth than current search
            else:
                transposition = self.table[key]
                hit = False
                score = None # Score in TT not useful
                bestmove = transposition['bestmove']  # Return best move to improve move ordering
                return (hit, score, bestmove)
        else: # Transposition not found
            hit = False
            score = None
            bestmove = ()
            return (hit, score, bestmove)
        
    def count_entry(self):
        return len(self.table.keys())

# [OUTDATED][MUST RUN] ttalphabeta()

In [3]:
# search function - alphabeta + transposition table
# dependencies
import numpy as np
import copy
import random
from chooseEval import evaluateScore
#from transpositiontable import TranspositionTable


def ttalphabeta(board, depth, ntype, p, a=-np.inf, b=np.inf, tt=TranspositionTable()):
    """
    Alpha-Beta search algorithm, to be used with iterationdeepening() and custom class TranspositionTable.
    All debug printouts suppressed.
    Parameters: 
        board (HexBoard object): 
        depth (int): depth limit of search tree, if depth exceeds empty positions, it will be reduced
        ntype (str): node type, etiher 'MAX' or 'MIN'
        p (int): perspective/player of search tree root, either 1 for HexBoard.BLUE, or 2 for HexBoard.RED
        a (float): alpha value, first input should be -np.inf or very small value, increase upon recursion
        b (float): beta value, first input should be np.inf or very large value, decrease upon recursion
        tt (TranspositionTable obj): initial value at root = {}
    Ouputs: 
        node (dict): {'state', 'depth', 'children', 'type', 'score', 'move'}
    Further improvements:
        search statistics: nodes searched + cutoffs
    """
    # Movelist for current state
    movelist = board.get_allempty()

    # For small board and end game, depth limit == full depth
    if depth > len(movelist):
        print('WARNING: DEPTH is limited by empty positions in board => set to full depth search. \n')
        depth = len(movelist)
    
    # Initialize node
    n = {'state': board,
         'depth': depth,
         'children': {},
         'type': ntype}
    
    # Print intial node info
    #print(f'Start of {n["type"]} node DEPTH = {n["depth"]}')
    #print(f'_Is the state of node GAME OVER: {n["state"].game_over}')
    #if (depth != 0) and not (n['state'].is_game_over()):
    #    print(f'_PLAYER {p} to consider EMPTY positions: {movelist}')
    #print('\n')
    
    # Look up transposition table for node and depth d
    (tt_hit, tt_score, tt_bestmove) = tt.lookup(n, depth)
    #print(f'TT lookup returns hit: {tt_hit}, score: {tt_score}, best move: {tt_bestmove} \n')
    
    if tt_hit:  # Found transposition at >= current search depth, copy and return TT result
        n['type'] = 'TT : ' + n['state'].convert_key()
        n['score'] = tt_score
        n['move'] = tt_bestmove
        #print('Found transposition at >= current search depth, copy and return TT result. \n')
        return (n, tt)
    
    # Update move list to search best move in TT first
    if tt_bestmove in movelist:
        #print('Best move is found in TT. Improve move ordering:')
        #print(f'Original movelist: {movelist}')
        movelist.remove(tt_bestmove)  # Remove best move in movelist
        movelist.insert(0, tt_bestmove)  # Insert best move to the first of movelist
        #print(f'New movelist: {movelist} \n')
    
    # Main loop
    if n['state'].is_game_over():  # Case: gameover at depth >= 0 (do we need to give bonus or penalty to score?)
        #print('This is a LEAF node')
        n['type'] = 'LEAF'  # Leaf node, Terminal node, no more child because game is over
        n['score'] = evaluateScore(n['state'], 'random')
        n['move'] = ()  # Store empty () to TT and return
        #print(f'Report SCORE for LEAF node: {n["score"]} \n')
        # Store n to TT and return n
    
    elif depth == 0:  # Case: reaching the search tree depth limit
        #print('This is a node at DEPTH==0')
        n['type'] = 'DEPTH==0' 
        n['score'] = evaluateScore(n['state'], 'random')
        n['move'] = ()  # Store empty () to TT and return
        #print(f'Report SCORE for node DEPTH==0: {n["score"]} \n')
        # Store n to TT and return n
    
    elif n['type'] == 'MAX':  # Max node
        #print('This is a MAX node \n')
        g_max = -np.inf  # Initialize max score with very small
        n['score'] = g_max
        for child_move in movelist:  # Search children / subtree
            #print(f'From DEPTH {n["depth"]} branch --> Child #{movelist.index(child_move)}: \n_PLAYER {p} will make move {child_move}')
            new_state = copy.deepcopy(n['state'])  # Copy state to aviod modifying node state
            new_state.place(child_move, p)  # Generate child state
            #print('_BEFORE move (current state):')
            #n['state'].print()            
            #print('_AFTER move (child state):')
            #new_state.print()
            #print('\n')
            new_p = [1, 2]
            new_p.remove(p)  # Reverse persective for child node
            (child_n, tt) = ttalphabeta(new_state, n['depth'] - 1, 'MIN', new_p[0], a, b, tt)  # Search OR evaluate child node, update TT
            n['children'].update({str(child_move): child_n})  # Store children node to current node
            if child_n['score'] > g_max:  # Update current node to backtrack from the maximum child node
                g_max = child_n['score']  # Update max score
                n['score'] = child_n['score']  # Store to return
                n['move'] = child_move  # Store to return
                a = max(a, g_max)  # Update alpha, traces the g_max value among siblings
            #print(f'End of child #{movelist.index(child_move)} move {child_move} for PLAYER {p} {n["type"]} node at DEPTH {n["depth"]}:', end=" ")
            #print(f'child score = {child_n["score"]}; Updated optimal move {n["move"]} has score = {n["score"]}. \n')
            #print(f'Bounds: alpha = {a} beta = {b} \n')
            if a >= b: # Check Beta cutoff
                #print(f'Beta cutoff takes place at move {child_move}; at child {movelist.index(child_move)};', end=" ")
                #print(f'pruning {len(movelist) - movelist.index(child_move)} out of {len(movelist)} children \n')
                break # Beta cutoff, stop searching other sibling
                
    elif n['type'] == 'MIN':  # Min node
        #print('This is a MIN node \n')
        g_min = np.inf  # Initialize min score with very large
        n['score'] = g_min
        for child_move in movelist:
            #print(f'From DEPTH {n["depth"]} branch --> Child #{movelist.index(child_move)}: \n_PLAYER {p} will make move {child_move}')
            new_state = copy.deepcopy(n['state'])
            new_state.place(child_move, p)          
            #print('_BEFORE move (current state):')
            #n['state'].print()            
            #print('_AFTER move (child state):')
            #new_state.print()
            #print('\n')
            new_p = [1, 2]
            new_p.remove(p)
            (child_n, tt) = ttalphabeta(new_state, n['depth'] - 1, 'MAX', new_p[0], a, b, tt)  # Child of MIN becomes MAX
            n['children'].update({str(child_move): child_n})
            if child_n['score'] < g_min:  # Update current node to backtrack from the minimum child node
                g_min = child_n['score']
                n['score'] = child_n['score']
                n['move'] = child_move
                b = min(b, g_min)  # Update beta, traces the g_min value among siblings             
            #print(f'End of child #{movelist.index(child_move)} move {child_move} for PLAYER {p} {n["type"]} node at DEPTH {n["depth"]}:', end=" ")
            #print(f'child score = {child_n["score"]}; Updated optimal move {n["move"]} has score = {n["score"]}. \n')
            #print(f'Bounds: alpha = {a} beta = {b} \n')
            if a >= b:
                #print(f'Alpha cutoff takes place at move {child_move}; at child {movelist.index(child_move)};', end=" ")
                #print(f'pruning {len(movelist) - movelist.index(child_move)} out of {len(movelist)} children \n')
                break  # Alpha cutoff, stop searching other sibling
    else: 
        print('SEARCH ERROR: Node type is unknown')
        return

    tt.store(n)  # Store search result of this node (state) to TT, and return
    #print(f'TT stored; Total # of entries in TT = {tt.count_entry()} \n')
    
    return (n, tt)


# Test cell for class: checking different conditions for tt.store()

In [179]:
fake_node = {'state': HexBoard(5), # empty board, different size, should not exist in TT
             'depth': result_node['depth'],
             'score': 99, # Very large score
             'move': (0,0)} # different move

In [180]:
tt.store(fake_node)
tt.table['232311233'] # Entry for root node in TT

{'state': <HexBoard.HexBoard at 0x1d6a9eaba90>,
 'depth': 1,
 'bestmove': (2, 2),
 'score': 6}

In [171]:
fake_node = {'state': result_node['state'],
             'depth': result_node['depth'] - 1, # shallow depth, expect no update
             'score': 99, # Very large score
             'move': (0,0)} # different move

In [172]:
tt.store(fake_node)
tt.table['232311233'] # Entry for root node in TT

{'state': <HexBoard.HexBoard at 0x1d6a9eaba90>,
 'depth': 1,
 'bestmove': (2, 2),
 'score': 6}

In [173]:
fake_node = {'state': result_node['state'],
             'depth': result_node['depth'], # same/larger depth, expect update
             'score': 99, # Very large score
             'move': (0,0)} # different move

In [175]:
tt.store(fake_node)
tt.table['232311233'] # Entry for root node in TT

{'state': <HexBoard.HexBoard at 0x1d6a9eaba90>,
 'depth': 1,
 'bestmove': (0, 0),
 'score': 99}

There is the dir(theobject) method to list all the fields and methods of your object (as a tuple) and the inspect module (as codeape write) to list the fields and methods with their doc (in """).

In [99]:
dir(board)

['BLUE',
 'EMPTY',
 'RED',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'board',
 'border',
 'check_win',
 'game_over',
 'get_allempty',
 'get_color',
 'get_neighbors',
 'get_opposite_color',
 'is_color',
 'is_empty',
 'is_game_over',
 'place',
 'print',
 'size',
 'traverse']

# Test alphabeta ID+TT

In [10]:
# Testing scenario

from HexBoard import HexBoard
# set up
board = HexBoard(4)
# place
board.place((0,0), 2)
board.place((2,1), 1)
board.place((0,2), 2)
board.place((1,1), 1)
board.place((2,0), 2)
board.print()
movelist = board.get_allempty()
print(f'Empty positions = {len(movelist)}: \n movelist: {movelist}')

   a b c d 
 -----------------------
0 |r - r - |
1 | - b b - |
2 |  r - - - |
3 |   - - - - |
   -----------------------
Empty positions = 11: 
 movelist: [(0, 1), (0, 3), (1, 0), (1, 2), (1, 3), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


In [11]:
random.seed(10)  # Set seed in case eval function use random

user_timelimit = 2  # USER please set time limit, start from small value

# call IDTT
(result_node, tt) = iterativedeepening(board=board, timelimit=user_timelimit, ntype='MAX', p=1)  # This tuple (result_node, tt) is for testing. Under normal run, TT is not returned.

USER NOTE: Interrupt the kernel to terminate search. 

[Iteration status] Start iteration at depth 1 

[Iteration status] Finish iteration at depth 1: Best move at root node = (0, 1) 

[Iteration status] Start iteration at depth 2 

[Iteration status] Finish iteration at depth 2: Best move at root node = (0, 3) 

[Iteration status] Start iteration at depth 3 

[Iteration status] Finish iteration at depth 3: Best move at root node = (1, 2) 

[Iteration status] Start iteration at depth 4 

[Iteration status] Finish iteration at depth 4: Best move at root node = (0, 3) 

[Iteration status] Start iteration at depth 5 

[Iteration status] Finish iteration at depth 5: Best move at root node = (0, 1) 

[Iteration status] Start iteration at depth 6 

[Iteration status] Finish iteration at depth 6: Best move at root node = (0, 3) 

[Iteration status] Start iteration at depth 7 

[Iteration status] Finish iteration at depth 7: Best move at root node = (0, 1) 

[Iteration status] Termination: TIM

In [12]:
# Size of TT
import sys
sys.getsizeof(tt.table)

147552

inspect result_node search tree

In [9]:
from printTree import printTree
printTree(result_node)

Search tree display format
 Depth D; Optimal move (x, y); Score <- Node type

D6  (0, 3)  7  <-  MAX
                         D5  (0, 1)  4  <-  MIN
                                                  D4  (0, 3)  4  <-  MAX
                                                                           D3  (3, 2)  1  <-  MIN
                                                                                                    D2  (1, 0)  1  <-  MAX
                                                                                                                             D1  (2, 3)  1  <-  MIN
                                                                                                                                                      D0  (1, 2)  4  <-  DEPTH==0
                                                                                                                                                      D0  (1, 3)  6  <-  DEPTH==0
                                                     